# https://www.youtube.com/watch?v=0dt0mujOgzY

In [1]:
from casadi import *


In [2]:
x = SX.sym('x')
z = SX.sym('z')
p = SX.sym('p')

In [3]:
dae =  dict(x=x,z=z, p=p, ode = z+p, alg =z*cos(z)-x)

In [6]:
F = integrator('F', 'idas', dae)

In [7]:
print(F)

F:(x0,z0,p,u[0],adj_xf[],adj_zf[],adj_qf[])->(xf,zf,qf[0],adj_x0[],adj_z0[],adj_p[],adj_u[]) IdasInterface


In [12]:
r = F(x0=0, z0=0, p=0.1)

In [13]:
print(r['xf'])

0.1724


In [15]:
x = SX.sym('x', 2)
p = SX.sym('p')
z = 1-x[1]**2
f = vertcat(z*x[0]-x[1]+p,x[0])


In [16]:
dae = dict(x=x, p=p, ode=f)

In [17]:
op=dict(t0=0, tf=1)
F = integrator('F', 'cvodes', dae, op)

CasADi - 2024-05-06 14:11:29 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:515]


In [18]:
r = F(x0=[0,1], p=0.1)

In [19]:
print(r['xf'])

[-0.922372, 0.551573]


In [20]:
a = SX.sym('a')
b =SX.sym('b')
sum1 = a+b
prod1 = a*b
F1 = Function('F1', [a,b], [sum1, prod1], ['a', 'b'], ['sx', 'xp'])

In [21]:
s1 = F1(a=2, b=3)

In [23]:
print(s1['sx'].full())

[[5.]]


In [30]:
# Diretc multiple shooting
T = 20. # Time horizon
N = 20 # Number of control intervals


# Declare model variables
x1 = MX.sym('x1')
x2 = MX.sym('x2')
x = vertcat(x1,x2)
u = MX.sym('u')

# Model equation
xdot = vertcat((1-x2**2)*x1 - x2+ u, x1)

# Objective term 
L= x1**2 + x2 **2 + u ** 2

# Formulate discrete time dynamics
if False:
    # CVODES from the SUNDIALS SUITE 

    #dae = dict(x=x, p=u, odex=xdot, quad=L) the same as 

    dae = {'x': x, 'p': u, 'ode':xdot, 'quad':L}
    opts = {'tf': T/N}
    F = integrator('F', 'cvodes', dae, opts)
else:
    # Fixed step Rune-Kutta 4 integrator
    M = 4
    DT = T/N/M
    f = Function('f', [x,u], [xdot, L])
    X0 = MX.sym('X0', 2)
    U = MX.sym('U')
    X = X0
    Q = 0
    for i in range(M):
        k1, k1_q = f(X,U)
        k2, k2_q = f(X + DT/2* k1, U)
        k3, k3_q = f(X + DT/2* k2, U)
        k4, k4_q = f(X + DT/2* k3, U)
        X = X+DT/6*(k1+2*k2+ 2*k3+k4)
        Q = Q + DT/6*(k1_q+2*k2_q+2*k3_q+k4_q)
    
    F= Function('F', [X0, U], [X, Q], ['x0','p'], ['xf', 'qf'])

# Evaluate at a test point
Fk = F(x0=[0.2,0.3], p = 0.4)
print(Fk['xf'])
print(Fk['qf'])



[0.417739, 0.622907]
0.467047


In [43]:
# Start with an empty NPL

w = []
w0 = []
lbw = []
ubw = []
J = 0 
g = []
ubg = []
lbg = []

# 'Lift' initial condictions
Xk = MX.sym('X0', 2)
w += [Xk]
lbw += [0,1]
ubw += [0,1]
w0 += [0,1]


In [44]:
# Formulate the NPL

for k in range(N):

    # new NLP variable for the control
    Uk = MX.sym('U_'+str(k))
    w += [ Uk]
    lbw += [-1]
    ubw  += [1]
    w0 += [0]

    # Integrate till end of the interval
    Fk = F(x0=Xk, p=Uk)
    Xk_end = Fk['xf']
    J = J+Fk['qf']

    # New NLP variable for state at end of interval
    Xk = MX.sym('X_'+str(k+1), 2)
    w += [Xk]
    lbw += [-0.25, -inf]
    ubw += [0, 0]

    # Add equality constraint 
    g += [Xk_end-Xk]
    lbg += [0, 0]
    ubg += [0, 0]


In [50]:
vertcat(*w)

MX(vertcat(X0, U_0, X_1, U_1, X_2, U_2, X_3, U_3, X_4, U_4, X_5, U_5, X_6, U_6, X_7, U_7, X_8, U_8, X_9, U_9, X_10, U_10, X_11, U_11, X_12, U_12, X_13, U_13, X_14, U_14, X_15, U_15, X_16, U_16, X_17, U_17, X_18, U_18, X_19, U_19, X_20))

In [49]:
# Create an NPL solver 

prob = {'f': J, 'x':vertcat(*w), 'g': vertcat(*g)}
solver = nlpsol('solver', 'ipopt', prob)

# Solve the NLP
sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
w_opt = sol['x'].ful().flatten()

# Plt the solution
x1_opt = w_opt[0::3]
x2_opt = w_opt[1::3]
u_pot = w_opt[2::3]

tgrid = [T/N*k for k in range(N+1)]
import matplotlib.pyplot as plt
plt.figure(1)
plt.clf()
plt.plot(tgrid, x1_opt, '--')
plt.plot(tgrid, x2_opt, '-')
plt.step(tgrid, vertcat(DM.nan(1), u_pot), '-.')
plt.xlabel('t')
plt.legend(['x1', 'x2', 'u'])
plt.grid()
plt.show()



RuntimeError: Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:1432:
Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:361:
.../casadi/core/function_internal.hpp:1649: Input 0 (x0) has mismatching shape. Got 22-by-1. Allowed dimensions, in general, are:
 - The input dimension N-by-M (here 62-by-1)
 - A scalar, i.e. 1-by-1
 - M-by-N if N=1 or M=1 (i.e. a transposed vector)
 - N-by-M1 if K*M1=M for some K (argument repeated horizontally)
 - N-by-P*M, indicating evaluation with multiple arguments (P must be a multiple of 1 for consistency with previous inputs)